# 形態素解析における辞書の説明

## 形態素解析の概説

__形態素解析__  
自然言語のテキストデータ（文）から、対象言語の文法や、辞書と呼ばれる単語の品詞等の情報にもとづき、  
形態素の列に分割= 分かち書きし、それぞれの形態素の品詞等を判別する作業である。  

__手法 __  
最近では「確率的言語モデルによる形態素解析」が主流になっている。
- ラティス上の経路予測
    - 事前に生成可能な単語列 (ラティス) を辞書を使って網羅的に列挙し、  各単語間の連結部において両単語が連結して出現する確率に相当するスコアを付与する。文全体でこのスコアの合計がもっとも高くなるような品詞列を答えとする。
    - ***辞書を使って 単語列を作成、コスト算出、品詞判定を行う。***
    - 統計的手法：隠れマルコフモデル（HMM）、条件付き確率場（CRF）  
    

- 点予測
    - 点予測ではすべての文字の境界に対し、単語単位に分割可能かどうか分類器により判定する。
    - ***単語のみ登録されている辞書で形態素解析可能*** 　（ただし解析に非常に時間がかかる。）
    - 統計的手法：サポートベクターマシン（SVM）、線形分離器　　
    
    
__辞書__  
単語の集約  
各単語に「単語の表層表現」「品詞」「活用形」「読み」「発音」「生起コスト（モデルにおける単語の生成しやすさ）」などの形態素解析に必要な情報が格納されている。  
→「コーパス」は文章単位の情報が格納されているもの

#### 辞書の中身（MeCabの場合）

In [ ]:
ジョージ,1291,1291,3632,名詞,固有名詞,人名,名,*,*,ジョージ,ジョージ,ジョージ
ブッシュ,1290,1290,4654,名詞,固有名詞,人名,姓,*,*,ブッシュ,ブッシュ,ブッシュ

左から、表層文字,左文脈ID,右文脈ID,コスト,品詞...

### フリーの形態素解析エンジン

・　ChaSen（茶筌）、chasen.naist.jp (ラティス上の経路予測、HMM)  
・ JUMAN、nlp.kuee.kyoto-u.ac.jp  
・　KAKASI（kanji kana simple inverter）、kakasi.namazu.org  
・　KyTea（キューティー）、http://www.phontron.com/kytea/ (点予測、線形分類)  
・　MeCab（和布蕪）、taku910.github.io/mecab (ラティス上の経路予測、CRF)  
・ JUMAN++  (Recurrent Neural Networkを使用・先月あたりに使用可能になった)

## MeCab

### MeCab概要

・ オープンソースの形態素解析エンジン  
・ 京都大学情報学研究科−日本電信電話株式会社コミュニケーション科学基礎研究所 共同研究ユニットプロジェクトで開発  
・ パラメータの推定に Conditional Random Fields (CRF) を用いている  
・ 奈良先端技術大学で開発された「ChaSen」を元に開発された  
・ ソフトウェアの仕様に加えC＃、Python、Java、R などへ組み込んで使うことも可能  
・ 各プログラミング言語への組み込みは専用のバインディングモジュールを仕様する。  
  　（PythonのものはWindowsだと非常に動かすまでに困難）

### MeCabの使い方

In [5]:
import MeCab
tagger = MeCab.Tagger('-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
print (tagger.parse('シンゴジラ'))

シンゴジラ	シンゴジラ	シン・ゴジラ	名詞-固有名詞-一般		
EOS



### MeCabの辞書

MeCabで仕様出来る辞書は、ざっと調べると以下の通り
- IPA辞書（IPAコーパス に基づき CRF でパラメータ推定した辞書）　
- Juman辞書（京都コーパスに基づき CRF でパラメータ推定した辞書）
- UniDic （国立国語学研究所のコーパス）
- NAIST (形態素解析用辞書 IPADIC の 表記ゆれ情報、複合語情報を付与した辞書)
- yahoo形態素解析辞書
- ipadic-neologd (twitterやwikipediaを元にコーパスを作成・GitHub上で随時更新)  
__ ＊ ユーザ辞書を作るカスタマイズも可能 __

### MeCabの実行しているラティス上の経路予測

#### 単語辞書
「ラティス上の経路予測」を用いた形態素解析では、まず全てのありうる結果を__「単語の辞書を用いて」__抽出する。  
つまり、__辞書にない単語__は__形態素解析する際に単語として認識されない__   


辞書内の全件を毎回検索していては計算時間が膨大にかかるため、以下のような特殊なデータの持ち方をしている。  
<img src="単語辞書.png">  

一文字目、二文字目とたどっていけるような構造。  
（TRIEというデータ構造の「ダブル配列」で実装している。）
白丸は次がある場合、赤丸は単語の終了として意味が通じるもの。  
上図例だと、一文字目が「あ」の場合は、「あわ」、「あら」、「あらし」、「あらまき」を辞書として持っている。

#### 形態素ラティス
形態素ラティスとは、前述の辞書を用いて抽出したすべてのありうる分かち書きの結果をグラフで表現した構造のこと。
<img src="形態素ラティス.png">  
例えば「東京都に住む」であれば、形態素ラティスは上図のようになる。  
ありうる分かち書きの結果を全てグラフで表現したもの。

#### 最小コスト法
形態素ラティスの中でどれが一番正しいと思われる分かち書きの仕方なのかを判断するために用る手法。  
コストとは具体的には、  
  ①二つの単語のつながりにくさ（連接コスト）  
  ②一つの単語の出現しやすさ（生起コスト）  
これらを各単語の辞書とともに持つことで、計算できるようにする。
<img src="最小コスト法.png">  
上図は、「東京都に住む」の形態素ラティスに対し、連接コストと生起コストを記してみたもの。  
上図では、「東京｜都｜に｜住む」のコストが
10+2+20+10+0+5+5+10+5=67（緑の枠でくくった数字の和）
となり、他と比べ最小なので、mecabの出力は「東京｜都｜に｜住む」になる。　　

東京|都|に（動詞）|住む           ：　10+2+20+10+5+30+10+10+5=102   
東｜京｜都｜に｜住む                  ：　10+5+10+5+20+10+0+5+5+10+5=85  
東｜京｜都｜に（動詞）｜住む    ：　10+5+10+5+20+10+5+30+10+10+5=120  
東｜京都｜に｜住む                     ：　10+5+10+5+20+5+5+10+5=75  
東｜京都｜に（動詞）｜住む       ：　10+5+10+5+10+30+10+10+5=95  

連接コスト・世紀コストを算出するためのモデル化にCRFが用いられている。    
（機械学習における『系列ラベリング問題』を解いている）  
コストが最小の経路を求める方法は「動的計画法」  

### 参考文献

- http://mieru-ca.com/blog/research-basics-of-text-mining/　　　
- http://chasen.naist.jp/chaki/t/2009-09-30/doc/mecab-cabocha-nlp-seminar-2009.pdf